# Infosys Quarterly Report Analysis

#### Developed By: Manaranjan Pradhan
#### www.manaranjanp.com

*This Jupyter notebook is confidential and proprietary to Manaranjan Pradhan. It is intended solely for authorized training purposes. Unauthorized distribution, sharing, or reproduction of this notebook or its contents is strictly prohibited. This material is for personal learning within the training program only and may not be used for commercial purposes or shared with others. Unauthorized use may result in disciplinary action or legal consequences. If you have received this notebook without authorization, please contact manaranjan@gmail.com immediately and delete all copies.*

In [1]:
!pip install getpass4 pypdf openai faiss-cpu llama-index llama-index-readers-file llama-index-vector-stores-faiss llama-index-embeddings-openai llama-index-llms-groq llama-index-embeddings-huggingface

  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of llama-cloud-services to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of llama-cloud-services to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 59.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 104.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.3/303.3 kB 20.7 MB/s eta 0:00:00
 

In [1]:
import nest_asyncio

nest_asyncio.apply()

In [2]:
import os
from getpass import getpass
os.environ["GROQ_API_KEY"] = getpass("Enter your Groq API key: ")

Enter your Groq API key: ··········


In [3]:
from llama_index.core import SimpleDirectoryReader, ServiceContext, VectorStoreIndex, StorageContext
from llama_index.core.response.pprint_utils import pprint_response
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.query_engine import SubQuestionQueryEngine
from llama_index.core.node_parser import SimpleNodeParser
from llama_index.core.node_parser import (SentenceWindowNodeParser,)
from llama_index.core.text_splitter import SentenceSplitter
from llama_index.core import Document
import faiss
from llama_index.vector_stores.faiss import FaissVectorStore
from llama_index.llms.groq import Groq
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

In [4]:
import faiss

## Configure LLM service

In [5]:
#llm = OpenAI(temperature=0,
#             model="chatgpt-4o-latest",
#             max_tokens=500)


llm = Groq(model="llama-3.3-70b-versatile")

In [6]:
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-large-en-v1.5")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/779 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [7]:
from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = embed_model
Settings.node_parser = SentenceSplitter(chunk_size=512, chunk_overlap=20)
Settings.num_output = 512
Settings.context_window = 3900

## Load data
Downloaded from

https://www.infosys.com/investors/reports-filings/quarterly-results.html

In [8]:
!pip install docling

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 7.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.7/223.7 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.2/164.2 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.1/15.1 MB 56.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 69.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.7/42.7 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 77.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 25.7 MB/s eta 0:00:00
   ━━━━

In [9]:
from docling.document_converter import DocumentConverter

In [12]:
def convert(file, ofile):
  converter = DocumentConverter()
  result = converter.convert(file)
  result.document.export_to_markdown()

  # Define the output directory
  output_dir = "output_documents"
  os.makedirs(output_dir, exist_ok=True)  # Ensure the directory exists

  # Define output file path
  output_path = os.path.join(output_dir, ofile)

  # Save as Markdown
  with open(output_path, "w", encoding="utf-8") as f:
      f.write(result.document.export_to_markdown())

  print(f"Document saved at: {output_path}")

In [14]:
convert("/content/ifrs-inr-press-release_q1_2022.pdf", "/content/ifrs-inr-press-release_q1_2022.md")
convert("/content/ifrs-inr-press-release_q1_2023.pdf", "/content/ifrs-inr-press-release_q1_2023.md")
convert("/content/ifrs-inr-press-release_q1_2024.pdf", "/content/ifrs-inr-press-release_q1_2024.md")

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Document saved at: /content/ifrs-inr-press-release_q1_2022.md


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Document saved at: /content/ifrs-inr-press-release_q1_2023.md


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Document saved at: /content/ifrs-inr-press-release_q1_2024.md


In [15]:
documents = [
    {
        "file_path": "/content/ifrs-inr-press-release_q1_2022.md",
        "metadata": {"year": "2022", "quarter": "Q1", "company": "Infosys"}
    },
    {
        "file_path": "/content/ifrs-inr-press-release_q1_2023.md",
        "metadata": {"year": "2023", "quarter": "Q1", "company": "Infosys"}
    },
    {
        "file_path": "/content/ifrs-inr-press-release_q1_2024.md",
        "metadata": {"year": "2024", "quarter": "Q1", "company": "Infosys"}
    }
]

In [16]:
nodes = []
parser = SimpleNodeParser()

for doc in documents:
    reader = SimpleDirectoryReader(input_files=[doc["file_path"]])
    data = reader.load_data()

    # Attach metadata
    for d in data:
        d.metadata = doc["metadata"]

    nodes.extend(parser.get_nodes_from_documents(data, show_progress=True))

print(f"Loaded {len(nodes)} nodes from documents")

Parsing nodes:   0%|          | 0/1 [00:00<?, ?it/s]

Parsing nodes:   0%|          | 0/1 [00:00<?, ?it/s]

Parsing nodes:   0%|          | 0/1 [00:00<?, ?it/s]

Loaded 18 nodes from documents


# Build indices

In [17]:
# dimensions of text-ada-embedding-002
d = 1024
faiss_index = faiss.IndexFlatL2(d)

In [18]:
vector_store = FaissVectorStore(faiss_index=faiss_index)
storage_context = StorageContext.from_defaults()
all_index = VectorStoreIndex(nodes, storage_context=storage_context)

## Build query engines

In [19]:
q_engine = all_index.as_query_engine(similarity_top_k=5)

## Run queries

In [20]:
response = q_engine.query(
    "What is the QoQ revenue growth in Q1 FY24?"
)

In [21]:
print(response)

To calculate the quarter-over-quarter (QoQ) revenue growth in Q1 FY24, we first need to establish the revenues for the relevant quarters. However, the provided data does not give us the revenues for Q1 FY24 or Q4 FY23. 

Given the data for the 3 months ended June 30, 2023, and June 30, 2022, as 27,896 and 23,665, respectively, for the previous year, we can see that the question's context has shifted to a different year's data. The question asks for Q1 FY24, but the data provided is for different periods.

Since the specific revenue figures for Q1 FY24 and Q4 FY23 are not provided, we cannot calculate the QoQ revenue growth for Q1 FY24. The information given does not allow for the direct calculation of QoQ growth for the specified period.


In [33]:
response = q_engine.query("Can you compare the operating margins over First quarters in FY22, FY23 and FY24 in infosys")
print(response)

To compare the operating margins over the first quarters in FY22, FY23, and FY24 for Infosys, we first need to calculate the operating margins for each of these periods.

The operating margin is calculated as the operating profit divided by the revenues for the period. However, the revenues for the periods are not directly provided in the given context. 

For the given periods, the operating profits are:
- FY23: 6,603
- Previous year: 5,365

Given the available data, we can calculate the operating margins if the revenues are known. However, the revenues for the first quarters of FY22, FY23, and FY24 are not provided in the context.

Without the specific financial data for revenues, we cannot accurately calculate the operating margins for these periods. The comparison between the given operating profits shows an increase from the previous year to FY23, but the exact operating margins cannot be determined without revenue data.


In [34]:
response = q_engine.query("Can you compare the assets over First quarters in FY22, FY23 and FY24 in infosys")
print(response)

To compare the assets over the first quarters in FY22, FY23, and FY24 for Infosys, we need to examine the total assets for each period. 

Given the provided data, the total liabilities and equity for the two periods were 109,486 and 108,386, respectively. The breakdown of equity includes total equity attributable to equity holders of the company and non-controlling interests. 

For the first quarter of FY24, the revenue growth was 4.2% year-over-year, and the operating margin was 20.8%. However, without more detailed information on the assets for Q1 FY24 or the previous years, we cannot accurately compare the total assets across the three quarters. 

The company's financial performance, including revenue growth and operating margins, provides insight into its overall financial health but does not directly inform a comparison of total assets across the specified periods.
